In [5]:
# -*- coding: utf-8 -*-
import cv2
import sys
from constants import *
from emotion_recognition import EmotionRecognition
from os.path import join
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import pandas
from pandas import DataFrame
from skimage.transform import resize

In [6]:
%matplotlib inline
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")

In [7]:
#Load file in folder
dirname = 'RafD'

#Load every image file in the provided directory
filenames = [os.path.join(dirname, fname)
             for fname in os.listdir(dirname)]

In [49]:
#Find Happy face only

import glob
happy = glob.glob('RafD/*happy*')
sad = glob.glob('RafD/*sad*')

#Make sure we have exactly 10 image files for test (you can include more)
happy = happy[:16]
sad = sad[:16]
assert(len(happy) == 16)
assert(len(sad) == 16)

In [51]:
# Read every filename as an RGB image
imgs_happy = [plt.imread(fname)[..., :3] for fname in happy]
imgs_sad = [plt.imread(fname)[..., :3] for fname in sad]

In [52]:
def imcrop_tosquare(img):
    """Make any image a square image.
    Parameters
    ----------
    img : np.ndarray
        Input image to crop, assumed at least 2d.
    Returns
    -------
    crop : np.ndarray
        Cropped image.
    """
    size = np.min(img.shape[:2])
    extra = img.shape[:2] - size
    crop = img
    for i in np.flatnonzero(extra):
        crop = np.take(crop, extra[i] // 2 + np.r_[:size], axis=i)
    return crop

def montage(images, saveto='montage.png'):
    """Draw all images as a montage separated by 1 pixel borders.
    Also saves the file to the destination specified by `saveto`.
    Parameters
    ----------
    images : numpy.ndarray
        Input array to create montage of.  Array should be:
        batch x height x width x channels.
    saveto : str
        Location to save the resulting montage image.
    Returns
    -------
    m : numpy.ndarray
        Montage image.
    """
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    if len(images.shape) == 4 and images.shape[3] == 3:
        m = np.ones(
            (images.shape[1] * n_plots + n_plots + 1,
             images.shape[2] * n_plots + n_plots + 1, 3)) * 0.5
    else:
        m = np.ones(
            (images.shape[1] * n_plots + n_plots + 1,
             images.shape[2] * n_plots + n_plots + 1)) * 0.5
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                m[1 + i + i * img_h:1 + i + (i + 1) * img_h,
                  1 + j + j * img_w:1 + j + (j + 1) * img_w] = this_img
    plt.imsave(arr=m, fname=saveto)
    return m

In [56]:
#Cropping, resize and convert to 4d array

def img_preprocessing( img_name ):
    # Crop every image to a square
    imgs = [imcrop_tosquare(img_i) for img_i in img_name]

    # Then resize the square image to 48 x 48 pixels
    imgs = [resize(img_i, (48, 48)) for img_i in imgs]

    # Make list of 3d imgaes a 4d array with the first dimension the number of images:
    imgs = np.array(imgs).astype(np.float32)
    return imgs

happy_pre = img_preprocessing(imgs_happy)
sad_pre = img_preprocessing(imgs_sad)

In [59]:
#TODO

happy_label
sad_label

(16, 48, 48, 3)

In [12]:
# Plot the resulting dataset:
# Make sure you "run" this cell after you create your `imgs` variable as a 4-D array!

plt.figure(figsize=(10, 10))
plt.imshow(montage(imgs, saveto='dataset.png'))

RuntimeError: Could not create write struct

In [34]:
import tflearn.datasets.mnist as mnist

X, Y, testX, testY = mnist.load_data(one_hot=True)


Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz


/home/ryan/anaconda2/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
/home/ryan/anaconda2/lib/python2.7/site-packages/tflearn/datasets/mnist.py:52: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz


array([[[[ 0.83167124,  0.83792126,  0.83877909],
         [ 0.84569162,  0.84569162,  0.84569162],
         [ 0.84860218,  0.85168886,  0.85014552],
         ..., 
         [ 0.86470586,  0.87220436,  0.86830962],
         [ 0.86449909,  0.86449909,  0.85665596],
         [ 0.86652881,  0.85976946,  0.84968597]],

        [[ 0.83739662,  0.83864123,  0.82776117],
         [ 0.83940333,  0.84754133,  0.83767998],
         [ 0.85582876,  0.84887028,  0.85040975],
         ..., 
         [ 0.8735562 ,  0.87139249,  0.85503983],
         [ 0.85882354,  0.85642999,  0.85030252],
         [ 0.86188728,  0.85955882,  0.84877449]],

        [[ 0.83968288,  0.84301472,  0.83350569],
         [ 0.84332871,  0.85092676,  0.84712774],
         [ 0.85821843,  0.85110295,  0.85442322],
         ..., 
         [ 0.86285996,  0.86298639,  0.85478324],
         [ 0.85789675,  0.85808825,  0.85793889],
         [ 0.85103017,  0.85090762,  0.84306449]],

        ..., 
        [[ 0.83784086,  0.85235143,

In [28]:
example_pic = imgs[0].shape
np_pic = np.array(example_pic)
np_pic

array([48, 48,  3])

In [32]:
image = tf.placeholder(tf.float32)

result = tf.add(image, 0)

h1 = layer(x,w)
h2 = layer(h1,w)
h3 = layer(h2,w)
O = layer(h3, w)

loss = lossfunction( O, label)
train = Optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run( result, feed_dict={x: Image, y= Label})

In [ ]:
image = tf.placeholder(tf.float32)

result = tf.add(image, 0)

with tf.Session() as sess:
    sess.run( result, feed_dict={image: np_pic})

In [59]:
s = pandas.DataFrame(imgs)

ValueError: Must pass 2-d input

In [4]:
network = EmotionRecognition()
network.build_network()

images = np.load('./data_kike.npy')
labels = np.load('./labels_kike.npy')

#images = images.reshape([-1, SIZE_FACE, SIZE_FACE, 1])
#labels = labels.reshape([-1, len(EMOTIONS)])

[+] Building CNN


InternalError: Dst tensor is not initialized.
	 [[Node: Momentum_2/zeros_7 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [18432,3072] values: 0 0 0...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
Caused by op u'Momentum_2/zeros_7', defined at:
  File "/home/ryan/anaconda2/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ryan/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/home/ryan/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in st

In [19]:
images.shape[0]

13975

In [25]:
data = np.zeros((len(EMOTIONS),len(EMOTIONS)))
for i in xrange(images.shape[0]):
    result = network.predict(images[i])
    data[np.argmax(labels[i]), result[0].index(max(result[0]))] += 1

In [31]:
for i in range(len(data)):
    total = np.sum(data[i])
    for x in range(len(data[0])):
        data[i][x] = data[i][x] / total

print(data)

[[ 0.          0.          0.          0.06584821  0.          0.93415179
   0.        ]
 [ 0.          0.          0.          0.04290429  0.          0.95709571
   0.        ]
 [ 0.          0.          0.          0.04761905  0.          0.95238095
   0.        ]
 [ 0.          0.          0.          0.03048327  0.          0.96951673
   0.        ]
 [ 0.          0.          0.          0.07432854  0.          0.92567146
   0.        ]
 [ 0.          0.          0.          0.03128689  0.          0.96871311
   0.        ]
 [ 0.          0.          0.          0.0678392   0.          0.9321608
   0.        ]]


In [34]:
print '[+] Generating graph'
c = plt.pcolor(data, edgecolors = 'k', linewidths = 4, cmap = 'Blues', vmin = 0.0, vmax = 1.0)
def show_values(pc, fmt="%.2f", **kw):
    from itertools import izip
    pc.update_scalarmappable()
    ax = pc.get_axes()
    ax.set_yticks(np.arange(len(EMOTIONS)) + 0.5, minor = False)
    ax.set_xticks(np.arange(len(EMOTIONS)) + 0.5, minor = False)
    ax.set_xticklabels(EMOTIONS, minor = False)
    ax.set_yticklabels(EMOTIONS, minor = False)
    for p, color, value in izip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha = "center", va = "center", color = color, **kw)

[+] Generating graph


In [35]:
show_values(c)
plt.xlabel('Predicted Emotion')
plt.ylabel('Real Emotion')
plt.show()

/home/ryan/anaconda2/lib/python2.7/site-packages/matplotlib/artist.py:221: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)
